In [1]:
# Packages used in the script
import numpy as np
import pandas as pd
from pandas.io.json import json_normalize
import io
import gzip
import spacy
import re
from contractions import contractions_dict

In [ ]:
df_sport = pd.read_csv(r'Path where the CSV file is stored\df_sports.csv')

In [11]:
df_sport['tokenize'] = df_sport['sentences'].apply(lambda x: [(token.text) for token in nlp(str(x)) if token.is_alpha if not token.like_url]) 

In [12]:
# Aspect extraction 
# Extract noun chunks from the answers
chunk = []
pos = []
 
def noun_chunk(doc):
    for row in doc:
        chunk.append([re.split(' ', str(x)) for x in nlp(str(row)).noun_chunks]) 
    for word in chunk:
        pos.append([(x.text, x.pos_) for x in nlp(str(word)) if x.is_alpha]) 
                    #if not x.pos_ == 'DET'
                    #if not x.pos_ == 'PRON'
                   # if not x.pos_ == 'ADP'
                    #if not x.pos_ == 'CCONJ'])
                    #if not x.pos_ == 'NUM'])
                        
    return pos
                 
df_sport['noun_chunk'] = noun_chunk(df_sport['sentences'])  


In [13]:
#Remove sentences without a noun_chunk
#Rows before removing empty noun_chunks 416821
df_sport = df_sport[df_sport['noun_chunk'].map(lambda d: len(d)) > 0].reset_index(drop = True)

In [14]:
df_sport['adjective'] = df_sport['noun_chunk'].apply(lambda x: [word for i in x for word in i if 'ADJ' in word])


In [15]:
df_sport = df_sport[df_sport['adjective'].map(lambda d: len(d)) > 0].reset_index(drop = True)

In [16]:
del df_sport['adjective']

In [17]:
df_sport

,sentences,product_type,tokenize,noun_chunk
0,the key requirement is that you must have a 1.5 ghz pentium 4 or higher (preferably dual or more cores).,0899334113,"[the, key, requirement, is, that, you, must, have, a, ghz, pentium, or, higher, preferably, dual, or, more, cores]","[(the, DET), (key, NOUN), (requirement, NOUN), (you, PRON), (a, DET), (ghz, NOUN), (pentium, NOUN), (preferably, ADV), (dual, ADJ), (or, CCONJ), (more, ADJ), (cores, NOUN)]"
1,note that this is a huge power-sucking program that will feel sluggish on less than about a 2.8 ghz p4 with a gig or two of ram.,0899334113,"[note, that, this, is, a, huge, power, sucking, program, that, will, feel, sluggish, on, less, than, about, a, ghz, with, a, gig, or, two, of, ram]","[(a, DET), (huge, ADJ), (power, NOUN), (sucking, VERB), (program, NOUN), (a, DET), (ghz, NOUN), (a, DET), (gig, NOUN), (ram, NOUN)]"
2,"you can turn off the 3d shaded effect (which effectively creates a clone of delorme street atlas - it is the identical road data) to dramatically speed up the scrolling, zooming and screen redraws.",0899334113,"[you, can, turn, off, the, shaded, effect, which, effectively, creates, a, clone, of, delorme, street, atlas, it, is, the, identical, road, data, to, dramatically, speed, up, the, scrolling, zooming, and, screen, redraws]","[(you, PRON), (the, DET), (shaded, ADJ), (effect, NOUN), (a, DET), (clone, NOUN), (delorme, VERB), (street, NOUN), (atlas, NOUN), (it, PRON), (the, DET), (identical, ADJ), (road, NOUN), (data, NOUN), (the, DET), (scrolling, VERB), (zooming, NOUN), (and, CCONJ), (screen, NOUN), (redraws, NOUN)]"
3,"you can purchase the map programs for a yearly fee and download usgs, color, satellite and other imagery.",0899334113,"[you, can, purchase, the, map, programs, for, a, yearly, fee, and, download, usgs, color, satellite, and, other, imagery]","[(you, PRON), (the, DET), (map, NOUN), (programs, NOUN), (a, DET), (yearly, ADJ), (fee, NOUN), (and, CCONJ), (download, NOUN), (usgs, NOUN), (color, NOUN), (satellite, NOUN), (other, ADJ), (imagery, NOUN)]"
4,"but i what i did not like about it was that you do not get a whole ""region"" like in the 3d topo quad maps",0899334113,"[but, i, what, i, did, not, like, about, it, was, that, you, do, not, get, a, whole, region, like, in, the, topo, quad, maps]","[(i, PRON), (what, PRON), (i, PRON), (it, PRON), (you, PRON), (a, DET), (whole, ADJ), (region, NOUN), (the, DET), (topo, NOUN), (maps, NOUN)]"
...,...,...,...,...
66893,you need to make sure you are able to locate the beam using a stud finder and by knocking on the ceiling to listen for a solid part which usually indicates a beam.,B00KZ41A1E,"[you, need, to, make, sure, you, are, able, to, locate, the, beam, using, a, stud, finder, and, by, knocking, on, the, ceiling, to, listen, for, a, solid, part, which, usually, indicates, a, beam]","[(you, PRON), (you, PRON), (the, DET), (beam, NOUN), (a, DET), (stud, NOUN), (finder, NOUN), (the, DET), (ceiling, NOUN), (a, DET), (solid, ADJ), (part, NOUN), (a, DET), (beam, NOUN)]"
66894,you can check out the 2014 installation video at http://youtu.be/dsm_frps_tkour and also other videos on our x-pole us youtube page which shows you how to set up the pole.,B00KZ41A1E,"[you, can, check, out, the, installation, video, at, and, also, other, videos, on, our, x, pole, us, youtube, page, which, shows, you, how, to, set, up, the, pole]","[(you, PRON), (the, DET), (installation, NOUN), (video, NOUN), (also, ADV), (other, ADJ), (videos, NOUN), (us, PRON), (you, PRON), (the, DET), (pole, NOUN)]"
66895,"you can actually see a fully detailed illustration of how it is assembled here: http://youtu.be/dsm_frps_tk also, if you have any additional questions please feel free to call x-pole at 888-976-5387.",B00KZ41A1E,"[you, can, actually, see, a, fully, detailed, illustration, of, how, it, is, assembled, here, also, if, you, have, any, additional, questions, please, feel, free, to, call, x, pole, at]","[(you, PRON), (a, DET), (fully

In [18]:
# List the positive and negative words
positive_words = []
negative_words = []

with open('positive-words.txt','r') as p:
    for line in p:
        for word in line.split():
            positive_words.append(word)

with open('negative-words.txt', 'r') as n:
    for line in n:
        for word in line.split():
            negative_words.append(word)

In [19]:
def opinion_word(x):
    y = []
    for word, tag in x:
        if tag == 'ADJ':
            if word in positive_words:
                y.append((word, 'P'))
            if word in negative_words:
                y.append((word, 'N'))
    return y    
                   
df_sport['opinion_word'] = df_sport['noun_chunk'].apply(opinion_word)

In [20]:
df_sport = df_sport[df_sport['opinion_word'].map(lambda d: len(d)) > 0].reset_index(drop = True)

In [246]:
df_sport

,sentences,product_type,tokenize,noun_chunk,opinion_word
0,i am not aware of any compatible software for ipad or ipad mini.,0899334113,"[i, am, not, aware, of, any, compatible, software, for, ipad, or, ipad, mini]","[(i, PRON), (any, DET), (compatible, ADJ), (software, NOUN), (ipad, NOUN), (or, CCONJ), (ipad, NOUN), (mini, NOUN)]","[(compatible, P)]"
1,removable exotic stripper dancing pole dance pole 45mm.....sold by honey-blue super outlet,6035000193,"[removable, exotic, stripper, dancing, pole, dance, pole, sold, by, honey, blue, super, outlet]","[(removable, ADJ), (exotic, ADJ), (stripper, NOUN), (dancing, NOUN), (pole, NOUN), (dance, NOUN), (pole, NOUN), (mm, NOUN), (honey, NOUN), (blue, ADJ), (super, ADJ), (outlet, NOUN)]","[(super, P)]"
2,"the band itself is 40"" long, then on each end there is a strong fabric connector with a d-link clasp which is 5 1/2"".",7245456313,"[the, band, itself, is, long, then, on, each, end, there, is, a, strong, fabric, connector, with, a, d, link, clasp, which, is]","[(the, DET), (band, NOUN), (itself, PRON), (each, DET), (end, NOUN), (a, DET), (strong, ADJ), (fabric, NOUN), (connector, NOUN), (a, DET), (link, VERB), (clasp, NOUN)]","[(strong, P)]"
3,"there is an attachment, actually a strong band of fabric with a loop, for the bands to go through, and the end has a thickened portion that can be placed on the other side of a door frame when the door is opened, but will not slip through when the door is closed.",7245456313,"[there, is, an, attachment, actually, a, strong, band, of, fabric, with, a, loop, for, the, bands, to, go, through, and, the, end, has, a, thickened, portion, that, can, be, placed, on, the, other, side, of, a, door, frame, when, the, door, is, opened, but, will, not, slip, through, when, the, door, is, closed]","[(an, DET), (attachment, NOUN), (actually, ADV), (a, DET), (strong, ADJ), (band, NOUN), (fabric, NOUN), (a, DET), (loop, NOUN), (the, DET), (bands, NOUN), (the, DET), (end, NOUN), (a, DET), (thickened, VERB), (portion, NOUN), (the, DET), (other, ADJ), (side, NOUN), (a, DET), (door, NOUN), (frame, NOUN), (the, DET), (door, NOUN), (the, DET), (door, NOUN)]","[(strong, P)]"
4,it is a good product.,7245456313,"[it, is, a, good, product]","[(it, PRON), (a, DET), (good, ADJ), (product, NOUN)]","[(good, P)]"
...,...,...,...,...,...
21471,good luck to you,B00KKWN7KS,"[good, luck, to, you]","[(good, ADJ), (luck, NOUN), (you, PRON)]","[(good, P)]"
21472,#1 - it's bpa free pet.,B00KOVQUK4,"[it, bpa, free, pet]","[(it, PRON), (bpa, NOUN), (free, ADJ), (pet, NOUN)]","[(free, P)]"
21473,it's the newest model and everything on the pole is updated for better use in general.,B00KZ41A1E,"[it, the, newest, model, and, everything, on, the, pole, is, updated, for, better, use, in, general]","[(it, PRON), (the, DET), (newest, ADJ), (model, NOUN), (everything, PRON), (the, DET), (pole, NOUN), (better, ADJ), (use, NOUN)]","[(better, P)]"
21474,you need to make sure you are able to locate the beam using a stud finder and by knocking on the ceiling to listen for a solid part which usually indicates a beam.,B00KZ41A1E,"[you, need, to, make, sure, you, are, able, to, locate, the, beam, using, a, stud, finder, and, by, knocking, on, the, ceiling, to, listen, for, a, solid, part, which, usually, indicates, a, beam]","[(you, PRON), (you, PRON), (the, DET), (beam, NOUN), (a, DET), (stud, NOUN), (finder, NOUN), (the, DET), (ceiling, NOUN), (a, DET), (solid, ADJ), (part, NOUN), (a, DET), (beam, NOUN)]","[(solid, P)]"


In [134]:
# Select random 4000 rows 
df_spo = df_sport.sample(n=5000, random_state = 2).reset_index(drop = True)

In [135]:
positive = []
negative = []

for chunk in df_spo['noun_chunk']:
    for word, tag in chunk:
        if word in positive_words:
            positive.append((word, tag))
        if word in negative_words:
            negative.append((word, tag))
            
print(len(positive))
print(len(negative))

5153
1644


In [136]:
# Aspect extraction positive aspects

words = []
tags = []


for sentence in df_spo['noun_chunk']:
    for word, tag in sentence:
        aspects_positive = []
        sentiment = []
        words.append(word)
        tags.append(tag)
        
        pattern1_pos = ['ADJ','NOUN']
              
        for p in pattern1_pos:
            if tag == 'ADJ':
                if word not in positive_words:
                    words.append(word)
                    tags.append(word)
        
        for i in range(len(tags)):
            if tags[i:i+len(pattern1_pos)] == pattern1_pos:
                aspects_positive.append(words[i:i+len(pattern1_pos)])
                
            
                 
        pattern2_pos = ['ADJ', 'NOUN', 'NOUN']            
        
        for p in pattern2_pos:
            if tag == 'ADJ':
                if word not in positive_words:
                    words.append(word)
                    tags.append(word)
                    
        for i in range(len(tags)):
            if tags[i:i+len(pattern2_pos)] == pattern2_pos:
                aspects_positive.append(words[i:i+len(pattern2_pos)])
                
            
                    
                    
        pattern3_pos = ['ADJ', 'NOUN', 'NOUN', 'NOUN']
        
        for p in pattern3_pos:
            if tag == 'ADJ':
                if word not in positive_words:
                    words.append(word)
                    tags.append(word)
            
        for i in range(len(tags)):
            if tags[i:i+len(pattern3_pos)] == pattern3_pos:
                aspects_positive.append(words[i:i+len(pattern3_pos)])
                
                
       
        pattern4_pos = ['ADJ', 'NOUN', 'NOUN', 'NOUN', 'NOUN']
        
        for p in pattern4_pos:
            if tag == 'ADJ':
                if word not in positive_words:
                    words.append(word)
                    tags.append(word)
            
        for i in range(len(tags)):
            if tags[i:i+len(pattern4_pos)] == pattern4_pos:
                aspects_positive.append(words[i:i+len(pattern4_pos)]) 
                
               
        
        pattern5_pos = ['ADJ', 'NOUN', 'NOUN', 'NOUN', 'NOUN', 'NOUN']
        
        for p in pattern5_pos:
            if tag == 'ADJ':
                if word not in positive_words:
                    words.append(word)
                    tags.append(word)
            
        for i in range(len(tags)):
            if tags[i:i+len(pattern5_pos)] == pattern5_pos:
                aspects_positive.append(words[i:i+len(pattern5_pos)]) 
                
                
    
        pattern6_pos = ['ADJ', 'VERB']
             
        for p in pattern6_pos:
            if tag == 'ADJ':
                if word not in positive_words:
                    words.append(word)
                    tags.append(word)    
                
        for i in range(len(tags)):
            if tags[i:i+len(pattern6_pos)] == pattern6_pos:
                aspects_positive.append(words[i:i+len(pattern6_pos)])
                
                        
aspects_positive = [' '.join(a) for a in aspects_positive]
sentiment = [' '.join(s) for s in sentiment]


In [137]:
# Remove duplicates 
aspects_positive = list(set(aspects_positive))

aspects_positive = [i for i in aspects_positive if len(i) >= 2]

In [243]:
print(len(aspects_positive))

3233


In [139]:
# Aspect extraction negative aspects

words = []
tags = []


for sentence in df_spo['noun_chunk']:
    for word, tag in sentence:
        aspects_negative = []
        sentiment = []
        words.append(word)
        tags.append(tag)
        
        pattern1_neg = ['ADJ','NOUN']
              
        for p in pattern1_neg:
            if tag == 'ADJ':
                if word not in negative_words:
                    words.append(word)
                    tags.append(word)
        
        for i in range(len(tags)):
            if tags[i:i+len(pattern1_neg)] == pattern1_neg:
                aspects_negative.append(words[i:i+len(pattern1_neg)])
                
                
                 
        pattern2_neg = ['ADJ', 'NOUN', 'NOUN']            
        
        for p in pattern2_neg:
            if tag == 'ADJ':
                if word not in negative_words:
                    words.append(word)
                    tags.append(word)
                    
        for i in range(len(tags)):
            if tags[i:i+len(pattern2_neg)] == pattern2_neg:
                aspects_negative.append(words[i:i+len(pattern2_neg)])
                
                
                    
                    
        pattern3_neg = ['ADJ', 'NOUN', 'NOUN', 'NOUN']
        
        for p in pattern3_neg:
            if tag == 'ADJ':
                if word not in negative_words:
                    words.append(word)
                    tags.append(word)
            
        for i in range(len(tags)):
            if tags[i:i+len(pattern3_neg)] == pattern3_neg:
                aspects_negative.append(words[i:i+len(pattern3_neg)])
                
                
       
        pattern4_neg = ['ADJ', 'NOUN', 'NOUN', 'NOUN', 'NOUN']
        
        for p in pattern4_neg:
            if tag == 'ADJ':
                if word not in negative_words:
                    words.append(word)
                    tags.append(word)
            
        for i in range(len(tags)):
            if tags[i:i+len(pattern4_neg)] == pattern4_neg:
                aspects_negative.append(words[i:i+len(pattern4_neg)]) 
                
                
        
        pattern5_neg = ['ADJ', 'NOUN', 'NOUN', 'NOUN', 'NOUN', 'NOUN']
        
        for p in pattern5_neg:
            if tag == 'ADJ':
                if word not in negative_words:
                    words.append(word)
                    tags.append(word)
            
        for i in range(len(tags)):
            if tags[i:i+len(pattern5_neg)] == pattern5_neg:
                aspects_negative.append(words[i:i+len(pattern5_neg)]) 
                
                
    
        pattern6_neg = ['ADJ', 'VERB']
             
        for p in pattern6_neg:
            if tag == 'ADJ':
                if word not in negative_words:
                    words.append(word)
                    tags.append(word)    
                
        for i in range(len(tags)):
            if tags[i:i+len(pattern6_neg)] == pattern6_neg:
                aspects_negative.append(words[i:i+len(pattern6_neg)])
                
                        
aspects_negative = [' '.join(a) for a in aspects_negative]
sentiment = [' '.join(s) for s in sentiment]

In [140]:
# Remove duplicates 
aspects_negative = list(set(aspects_negative))

aspects_negative = [i for i in aspects_negative if len(i) >= 2]

In [245]:
print(len(aspects_negative))

961


In [142]:
# Concatenate the two lists togehter
aspects = [y for x in [aspects_positive, aspects_negative] for y in x]

In [244]:
print(len(aspects))

4194


In [144]:
# Match the aspect words with the sentence 
df_spo['aspect_word'] = df_spo['sentences'].str.findall(r"\b"+"|".join(aspects) + r"\b").str.join(' ')

In [145]:
df_spo['aspect_word'] = df_spo['aspect_word'].str.split()

In [147]:
# IOB tagging
aspect = [x.split()[0:3] for x in aspects]
pos = [x.split()[0:3] for x in aspects_positive]
neg = [x.split()[0:3] for x in aspects_negative]
p = [x[0] for x in pos]
n = [x[0] for x in neg]   
beginning_chunk_positive = [x[1] for x in pos]
beginning_chunk_negative = [x[1] for x in neg]
remaining_chunk_positive = [x[-1] for x in pos]
remaining_chunk_negative = [x[-1] for x in neg]
#remaining_chunk = [x for x in remaining_words if x]


def IOB_tagging(sentence):  
    tags =[]
    opinion_p = []
    opinion_n = []
    chunk_p = []
    chunk_n = []
    
    for word in sentence:
        try:
            if word in p:
                opinion_p = word
                tags.append('O')
            elif word in beginning_chunk_positive and (word[-1] in opinion_p or word[1] in opinion_p):
                chunk_p = word
                tags.append('B_pos')
            elif word in remaining_chunk_positive and word[-2] in chunk_p:
                tags.append('I_pos')
            elif word in n:
                opinion_n = word
                tags.append('O')
            elif word in beginning_chunk_negative and (word[-1] in opinion_n or word[1] in opinion_n):
                chunk_n = word
                tags.append('B_neg')
            elif word in remaining_chunk_negative and word[-2] in chunk_n:
                tags.append('I_neg')
            else:
                tags.append('O')
        except IndexError:
            pass
    
    return tags

df_spo['IOB'] = df_spo['tokenize'].apply(IOB_tagging)

In [148]:
df_spo[0:50]

,sentences,product_type,tokenize,noun_chunk,opinion_word,aspect_word,IOB
0,i bought 2 that looked very similar one had a squeezy bottom half and the other is solid plastic .,B002VEDG7M,"[i, bought, that, looked, very, similar, one, had, a, squeezy, bottom, half, and, the, other, is, solid, plastic]","[(i, PRON), (one, NUM), (a, DET), (squeezy, NOUN), (bottom, NOUN), (solid, ADJ), (plastic, NOUN)]","[(solid, P)]","[solid, plastic]","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, B_pos]"
1,"if an adult steps on it they will just about sink to the ground, but for small children it is just enough tension.",B002N5AHHQ,"[if, an, adult, steps, on, it, they, will, just, about, sink, to, the, ground, but, for, small, children, it, is, just, enough, tension]","[(it, PRON), (they, PRON), (the, DET), (ground, NOUN), (small, ADJ), (children, NOUN), (it, PRON), (just, ADV), (enough, ADJ), (tension, NOUN)]","[(enough, P)]","[enough, tension]","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, B_pos]"
2,"well, i did not see the warning label, but i do know that california has some weird laws that you need to follow in order to sell some things there.",B0076D0MAC,"[well, i, did, not, see, the, warning, label, but, i, do, know, that, california, has, some, weird, laws, that, you, need, to, follow, in, order, to, sell, some, things, there]","[(i, PRON), (the, DET), (warning, NOUN), (label, NOUN), (i, PRON), (california, NOUN), (some, DET), (weird, ADJ), (laws, NOUN), (you, PRON), (order, NOUN), (some, DET), (things, NOUN)]","[(weird, N)]","[weird, laws]","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]"
3,happy healthy drinking;),B003DTLNX8,"[happy, healthy, drinking]","[(happy, ADJ), (healthy, ADJ), (drinking, VERB)]","[(happy, P), (healthy, P)]","[healthy, drinking]","[O, O, O]"
4,so i may not be the best person to answer.,B0055WRX30,"[so, i, may, not, be, the, best, person, to, answer]","[(i, PRON), (the, DET), (best, ADJ), (person, NOUN)]","[(best, P)]","[best, person]","[O, O, O, O, O, O, B_pos, O, I_pos]"
5,i just used it 3 days ago and got a decent sized fish.,B000KD7DIE,"[i, just, used, it, days, ago, and, got, a, decent, sized, fish]","[(i, PRON), (it, PRON), (a, DET), (decent, ADJ), (sized, ADJ), (fish, NOUN)]","[(decent, P)]","[decent, size]","[O, O, O, O, O, O, O, O, O, O, O]"
6,the pads are positioned properly and will prevent numb hands.,B009R7778K,"[the, pads, are, positioned, properly, and, will, prevent, numb, hands]","[(the, DET), (pads, NOUN), (numb, ADJ), (hands, NOUN)]","[(numb, N)]","[numb, hands]","[O, O, O, O, O, O, O, O, O, O]"
7,"the rifle features an adjustable stock for comfortable shooting & easy carrying, a 500 rd magazine and a shot velocity of up to 400 fps.",B00AAFSW6O,"[the, rifle, features, an, adjustable, stock, for, comfortable, shooting, easy, carrying, a, rd, magazine, and, a, shot, velocity, of, up, to, fps]","[(the, DET), (rifle, NOUN), (an, DET), (adjustable, ADJ), (stock, NOUN), (comfortable, ADJ), (shooting, VERB), (easy, ADJ), (carrying, VERB), (a, DET), (rd, NOUN), (magazine, NOUN), (a, DET), (shot, NOUN), (velocity, NOUN), (up, NOUN), (to, PART), (fps, NOUN)]","[(adjustable, P), (comfortable, P), (easy, P)]","[adjustable, stock, comfortable, shooting, easy, carrying]","[O, O, O, O, O, B_pos, O, O, O, O, B_pos, O, O, B_pos, O, O, O, O, O, O, O, O]"
8,"it was very easy, you just need to make sure the pedals are on the correct sides, s it does make a difference.",B001NGF69A,"[it, was, very, easy, you, just, need, to, make, sure, the, pedals, are, on, the, correct, sides, s, it, does, make, a, difference]","[(it, PRON), (you, PRON), (the, DET), (pedals, NOUN), (the, DET), (correct, ADJ), (sides, NOUN), (it, PRON), (a, DET), (difference, NOUN)]","[(correct, P)]","[correct, sides]","[O, O, O, O, O, O, O, O, O, O, O, B_pos, O, O, O, O, I_pos, O, O, O, O, B_pos]"
9,only coat me $3 and was a great fix thus far. haven't seen replacements personally.,B004

In [ ]:
# Export DataFrame to CSV file
df_spo.to_csv(r'/Users/annabellesonneveldt/downloads/df_sport.csv', index = False)